# Scraping Thơ Đỗ Phủ trên thivien.net
- lấy url của các bài thơ
- mở url lấy nội dung thơ (chữ Hán, phiên âm Hán Việt, phiên dịch)
*trang block scrap nên phải dùng header 

In [6]:
import requests
from bs4 import BeautifulSoup
import random
import csv
import re
import pandas 
import requests

from docx import Document

import time


In [20]:


user_agents_list = [
    'Mozilla/5.0 (iPad; CPU OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.83 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36'
]

# URL của trang web cần scrap
url = "https://www.thivien.net/%C4%90%E1%BB%97-Ph%E1%BB%A7/author-Se3x2jWTezvoDpkpSU9qjg"

try:
    # Tải nội dung của trang web
    print("Đang tải nội dung trang web...")
    response = requests.get(url, headers={'User-Agent': random.choice(user_agents_list)})
    response.raise_for_status()  # Ném ngoại lệ cho các mã trạng thái không tốt
    print("Nội dung trang web đã được tải thành công.")

    # Phân tích HTML content
    print("Phân tích HTML content...")
    soup = BeautifulSoup(response.content, "html.parser")

    # Trích xuất các URL từ trang web chứa từ "poem" nhưng không lấy từ "searchpoem"
    print("Trích xuất các URL chứa từ 'poem' nhưng không lấy từ 'searchpoem'...")
    urls = [a["href"] for a in soup.find_all("a", href=True) if "poem" in a["href"] and "searchpoem" not in a["href"] and "addpoem" not in a["href"] and "m-poems" not in a["href"]]


    # Lưu các URL vào tệp CSV
    file_path = "D:/Dophu-url-poem.csv"
    with open(file_path, "w", newline="", encoding="utf-8") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["URL"])

        for url in urls:
            writer.writerow([url])

    print(f"Các URL chứa từ 'poem' nhưng không lấy từ 'searchpoem' đã được lưu vào {file_path}")

except requests.exceptions.RequestException as e:
    print("Lỗi khi tải URL:", e)
except Exception as e:
    print("Đã xảy ra lỗi:", e)


Đang tải nội dung trang web...
Nội dung trang web đã được tải thành công.
Phân tích HTML content...
Trích xuất các URL chứa từ 'poem' nhưng không lấy từ 'searchpoem'...
Các URL chứa từ 'poem' nhưng không lấy từ 'searchpoem' đã được lưu vào D:/Dophu-url-poem.csv


In [ ]:

# Đọc file csv chứa các URL
df = pd.read_csv("D:/Dophu-url-poem2.csv")

# Hàm để thêm phần đầu của URL
def add_prefix(url):
    return "https://www.thivien.net" + url

# Thêm phần đầu vào từng URL
df['Full_URL'] = df['URL'].apply(add_prefix)

# Hàm để trích xuất dữ liệu từ URL và lưu vào file .docx
def extract_poem_data(url):
    user_agents_list = [
        'Mozilla/5.0 (iPad; CPU OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148',
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.83 Safari/537.36',
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36'
    ]
    HEADERS = {'User-Agent': 'Mozilla/5.0 (iPad; CPU OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148'}
    try:
        # Delay 15 giây trước khi tải nội dung trang web
        time.sleep(10)
        
        # Tải nội dung của trang web
        print("Đang tải nội dung trang web:", url)
        #response = requests.get(url, headers={'User-Agent': random.choice(user_agents_list)})
        response = requests.get(url, headers=HEADERS)
        response.raise_for_status()  # Ném ngoại lệ cho các mã trạng thái không tốt
   
        # Tạo biến đếm cho số lần đã mở link
        link_count = 0
        
        # Phân tích HTML content
        soup = BeautifulSoup(response.content, "html.parser")

        # Trích xuất tiêu đề
        title = soup.title.text.strip()
        print("Tiêu đề:", title)

        # Trích xuất phần văn bản chữ Hán và chữ Việt
        han_viet_text = ""
        for p_tag in soup.find_all('p', class_='HanChinese transcriptable'):
            han_viet_text += p_tag.text.strip() + "\n"

        # Tìm thẻ meta có nội dung mô tả và lấy nội dung của nó
        meta_description = soup.find("meta", {"name": "description"})
        if meta_description:
            description_content = meta_description["content"].strip()
        else:
            description_content = "Không tìm thấy thẻ meta mô tả"
            
        # Tìm tất cả các thẻ span có class là "gray"
        gray_spans = soup.find_all("span", class_="gray")

        # Lưu nội dung của từng thẻ span gray vào một list
        translations = []
        for gray_span in gray_spans:
            em_tag = gray_span.find("em")
            if em_tag:
                content_text = em_tag.text.strip()
                translations.append(content_text)
        
        # Lưu dữ liệu vào file .docx
        
        doc = Document()
        doc.add_heading(title, level=1)
                
        doc.add_paragraph("Chữ Hán:")
        han_viet_paragraph = doc.add_paragraph(han_viet_text)
  
                
        doc.add_paragraph("Phiên âm:")
        doc.add_paragraph(description_content)
                
        doc.add_paragraph("Phiên dịch:")
        for translation in translations:
            doc.add_paragraph(translation)
                
        doc.save("D:/dophu_poem.docx")
        print("Đã thêm dữ liệu mới vào file D:/dophu_poem.docx")
        # Tăng biến đếm sau mỗi lần mở link
        link_count += 1
        
        # Kiểm tra nếu biến đếm chia hết cho 5 thì nghỉ ngơi 10 giây
        if link_count % 5 == 0:
            print("Nghỉ ngơi 10 giây...")
            time.sleep(15)
            
    except requests.exceptions.RequestException as e:
        print("Lỗi khi tải URL:", e)
    except Exception as e:
        print("Đã xảy ra lỗi:", e)

# Gọi hàm để trích xuất dữ liệu từ từng URL và lưu vào file .docx
for url in df['Full_URL']:
    extract_poem_data(url)


Đang tải nội dung trang web: https://www.thivien.net/%C4%90%E1%BB%97-Ph%E1%BB%A7/Thu-h%E1%BB%A9ng-k%E1%BB%B3-1/poem-d-FDjEupVCgpsc1eaacNvg
Tiêu đề: Bài thơ: Thu hứng kỳ 1 - 秋興其一 (Đỗ Phủ - 杜甫)
Đã thêm dữ liệu mới vào file D:/dophu_poem.docx
Đang tải nội dung trang web: https://www.thivien.net/%C4%90%E1%BB%97-Ph%E1%BB%A7/%C4%90%C4%83ng-cao/poem-h3EvqeQvlQUxq2sceUWWYQ
Tiêu đề: Bài thơ: Đăng cao - 登高 (Đỗ Phủ - 杜甫)
Đã thêm dữ liệu mới vào file D:/dophu_poem.docx
Đang tải nội dung trang web: https://www.thivien.net/%C4%90%E1%BB%97-Ph%E1%BB%A7/Nguy%E1%BB%87t-d%E1%BA%A1/poem-T7ykhmCIqNvTZ_L2JmhFFw
Tiêu đề: Bài thơ: Nguyệt dạ - 月夜 (Đỗ Phủ - 杜甫)
Đã xảy ra lỗi: [Errno 13] Permission denied: 'D:/dophu_poem.docx'
Đang tải nội dung trang web: https://www.thivien.net/%C4%90%E1%BB%97-Ph%E1%BB%A7/Mao-%E1%BB%91c-v%E1%BB%8B-thu-phong-s%E1%BB%9F-ph%C3%A1-ca/poem-j4UAOMEqMgeNDayunsRjYA
Tiêu đề: Bài thơ: Mao ốc vị thu phong sở phá ca - 茅屋為秋風所破歌 (Đỗ Phủ - 杜甫)
Đã thêm dữ liệu mới vào file D:/dophu_poem.docx
Đ

# Phần Test khi chỉ lấy 1 bài thơ

In [10]:
import requests
from bs4 import BeautifulSoup

HEADERS = {'User-Agent': 'Mozilla/5.0 (iPad; CPU OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148'}

# URL của trang web cần lấy nội dung
url = "https://www.thivien.net/%C4%90%E1%BB%97-Ph%E1%BB%A7/%C4%90%E1%BB%91i-v%C5%A9-th%C6%B0-ho%C3%A0i-t%E1%BA%A9u-y%C3%AAu-H%E1%BB%A9a-th%E1%BA%ADp-nh%E1%BA%A5t-b%E1%BA%A1-c%C3%B4ng/poem-94BVxupMVXN25dDk5qXS2g"

try:
    # Tải nội dung của trang web
    response = requests.get(url, headers=HEADERS)
    response.raise_for_status()  # Ném ngoại lệ cho các mã trạng thái không tốt

    # Phân tích HTML content
    soup = BeautifulSoup(response.content, "html.parser")

    # Trích xuất tiêu đề
    title = soup.title.text.strip()
    print("Tiêu đề:", title)

    # Tìm thẻ meta có nội dung mô tả và lấy nội dung của nó
    meta_description = soup.find("meta", {"name": "description"})
    if meta_description:
        description_content = meta_description["content"].strip()
        print("Nội dung mô tả:")
        print(description_content)
    else:
        print("Không tìm thấy thẻ meta mô tả")

except requests.exceptions.RequestException as e:
    print("Lỗi khi tải URL:", e)
except Exception as e:
    print("Đã xảy ra lỗi:", e)


Tiêu đề: Bài thơ: Đối vũ thư hoài, tẩu yêu Hứa thập nhất bạ công - 對雨書懷走邀許十一簿公 (Đỗ Phủ - 杜甫)
Nội dung mô tả:
Đông nhạc vân phong khởi,
Dung dung mãn thái hư.
Chấn lôi phiên mạc yến,
Sậu vũ lạc hà ngư.
Toạ đối hiền nhân tửu,
Môn thinh trưởng giả xa.
Tương yêu quý nê ninh,
Kỵ mã đáo giai trừ.
